In [1]:
import sys
sys.path.insert(1, '/home/jupyter/crisp')

In [2]:
from synthetic.synthetic_generator import synthetic_generator
from models.Deconfounder import Deconfounder
from dataio.DataFrameDataset import *
from dataio.datasets import *

In [3]:
df = synthetic_generator()
df.head()
df.to_pickle("../data/test.pkl")

In [21]:
config = {
    "name": "Example Experiment for AH casual ensemble",
    "short_name": "ah_experiment_notebook",
    "bucket_project": "fdl-us-astronaut-health",
    "bucket_name": "ah_21_data",
    "bucket_path": "gs://ah_21_data",
    "verbose": 1,
    "test_val_split": [0.1, 0.1],
    "per_variant_experiment": False,
    "data_options": {
        'dataset_fp': '../data/test.pkl',
        'subject_keys': 'Subj_ID',
        'targets': ['Target'],
        'predictors': 'All',
        'environments': ['env_split'],
        'exclude': ['Subj_ID']
    },
    "feature_selection_options": {
        "max_features": 20,
        "verbose": 0,
        "seed": 12
    },
    "ensemble_options": {
        "models": ["ERM", "RF", "ICP", "IRM", "DCF", "ITE", "LIRM", "NLICP"]
    },
    "use_cloud":False,
    "results_directory": "results/"
}
data_config = config['data_options']

In [22]:
environment_datasets, val_dataset, test_dataset = get_datasets_for_experiment(config)

Running a per sample experiment
Loaded  2  train environments
Env  0  has  33  samples
X shape  (33, 350)  y shape  (33, 1)
Env  1  has  39  samples
X shape  (39, 350)  y shape  (39, 1)
Loaded val set, X shape: (11, 350)  y shape:  (11, 1)
Loaded test set, X shape: (17, 350)  y shape:  (17, 1)


In [ ]:
# Deconfounder args
dcf_args = {
    "minP": 0.1,
    "maxP": 0.9,
    "minFeatures": 1,
    "minAccuracy": 0.5,
    "seed": 0,
    "verbose": 1,
    "target": data_config['targets'],
    "output_pvals": True
}

In [29]:


# If we wish to output p vals for each feature, then use reduced feature set
# if dcf_args["output_pvals"] and (original_dimensionality > 500):
#     dcf_args["columns"] = reduced_test_dataset.predictor_columns
#     dcf = Deconfounder(reduced_environment_datasets,
#                        reduced_val_dataset,
#                        reduced_test_dataset,
#                        dcf_args)
# else:
dcf_args["columns"] = test_dataset.predictor_columns
dcf = Deconfounder(environment_datasets,
                   val_dataset,
                   test_dataset,
                   dcf_args)

dcf_results_dict = dcf.predictor_results()
print("Finished DCF")
# to_bucket = dcf_results_dict['to_bucket']
# to_bucket_results.append(to_bucket)


testshape: (17, 351)
valshape: (11, 351)
trainshape: (72, 351)
Number of features: 350
to drop: []
dropped 0 std columns of self.dfX
Number of features after correlated and 0 std features elimination: 325


/home/jupyter/crisp/models/Deconfounder.py:203: UserWarning: tfp.edward2 module is deprecated and will be removed on 2019-12-01. Use https://github.com/google/edward2 library instead.
  log_joint = ed.make_log_joint_fn(self.ppca_model)





Inferred W:
[[-3.95372361e-01 -2.24448442e-01  2.51546153e-03 -1.68690965e-01
   5.39427586e-02  2.34393895e-01  5.78158259e-01 -1.96531806e-02
  -2.12178186e-01  7.19708860e-01  3.19508195e-01 -1.12196051e-01
  -5.23787923e-02  5.96894622e-02 -1.55457212e-02 -1.96071684e-01
  -1.01341628e-01  4.21247274e-01  6.15848005e-01  2.47771531e-01
  -6.16442934e-02  8.03362057e-02  2.03414440e-01 -4.96024787e-01
   6.09136403e-01 -5.60005242e-03  1.11276639e+00  4.12562281e-01
   1.44766808e-01 -1.62560314e-01 -7.40626454e-02 -2.76383106e-02
  -3.84513259e-01  6.64321035e-02  6.53800219e-02  3.02834123e-01
   2.68350959e-01 -2.34347612e-01 -3.63491356e-01 -2.44251601e-02
   8.02312195e-02 -5.96324801e-01  9.65504814e-03  3.34761918e-01
   1.91124022e-01 -3.07992160e-01 -4.36576545e-01  1.32124126e-01
   2.49306094e-02  5.41624188e-01  2.85167664e-01  7.99020410e-01
   5.22365272e-01 -4.06512827e-01  5.63626960e-02  4.33021188e-01
   2.24850520e-01  3.65323037e-01  1.23043954e-01  3.73942792

In [33]:
from tensorflow_probability import edward2 as ed

In [32]:
ed.interception

TypeError: interception() missing 1 required positional argument: 'interceptor'